In [1]:
import csv
import copy
import math
import pandas as pd
from datetime import datetime

In [2]:
"""
FIRST OF ALL WE HAD TO USE THIS NON BEST PRACTICE METHODOLOGY SINCE THE LIMIT OF PYTHON RECURSIONS IS 1000
AND OUR DATASET RECORDS ARE MORE THAN 12000 SO IT'S NOT MY RESPONSIBILITY -_-
"""
import sys
sys.setrecursionlimit(13000) 

# 1 — remove_missing
def remove_missing(records, fields):

    if not records:
        return []
    
    head, *tail = records   # devides the `records` list into a bare record and a tail list by using *tail means to pack it into a list
    if non_empty_record(head, fields):
        return [head] + remove_missing(tail, fields)    # thats just concatenation of lists wiyth the record if it doesn't have empty values -_-
    else:
        return remove_missing(tail, fields)             # soo we are gonna skip this record if it has empty values -_-



def non_empty_record(record, fields):
        if not fields:
            return True
        head, *tail = fields
        value = record.get(head)    # same as record[head] but more safe to not get annoying errors
        if value in (None, "", "NA") or (isinstance(value, float) and math.isnan(value)):
            return False
        return non_empty_record(record, tail)   # instead of using FUNCTIONAL LOOPS now we are stuck in recursions -_-!


# 2 — fill_missing

def fill_missing(records, defaults):

    def fill_record(record, keys):      # JUST ONE RECORD OKE DON'T BE BLIND
        if not keys:
            return {}                   # thats a dict haa focus not a list, it has key:value pair
        key = keys[0]
        value = record.get(key)
        default = defaults.get(key)     # remember that this returns none if it didn't find it at this index
        if is_missing_value(value):
            filled_value = default
        else:
            filled_value = bool(value) if isinstance(default, bool) else value      # maybe its just 0 or 1 and the field is boolean so cast it 
        """
        remember that we cannot alter the variable since its functional paradime so its immutable, so we needed to do this COMPLEX way
        SO FOCUS VERY WELL, at first the recursion in the next line is built from left to right everytime passing the function the
        next element of the record and its key (column name) oke?, element1 with its key calls element2 with its key calls element3... complete by yourself -_-
        ofcourse at the end of the elements it hits the base function so we backpropagate so we return the `key:value` pair of each element
         from right to left BUT we need to concatenate elements inside one dictinory so we need a thing to unpack the dict and thats the ** of the
         `**rest_filled` job, if you need more info...easy you can search
        """
        rest_filled = fill_record(record, keys[1:])
        return {key: filled_value, **rest_filled}

    if not records:
        return []

    head = records[0]
    tail = records[1:]      # another way instead of using tail* IF ITS HARD FOR YOU TO UNDERSTAND
    return [fill_record(head, list(head.keys()))] + fill_missing(tail, defaults)



def is_missing_value(value):
        return value in (None, "", "NA") or (isinstance(value, float) and math.isnan(value))



# 3 — standardize_date
def standardize_date(record, field):
    """
    this function standardizes the date of the given field of the given record (exactly one record)
    """
    new_record = copy.deepcopy(record)
    try:
        parsed = datetime.strptime(new_record[field], "%m/%d/%Y")
        new_record[field] = parsed.strftime("%Y-%m-%d")
    except:
        pass
    return new_record


# 4 — standardize_dates
def standardize_dates(records, field):
    """
    this function does the same as the previous function but for multiple records
    """
    return list(map(lambda record: standardize_date(record, field), records))


# # 5 — normalize_numbers
def normalize_numbers(records, fields, precision):

    def normalize_record(items):
        if not items:
            return {}
        k, v = items[0]
        rest = normalize_record(items[1:])
        if k in fields:
            return {k: round(float(v), precision), **rest}
        else:
            return {k: v, **rest}

    if not records:
        return []

    head, *tail = records
    # `list(head.items()))` just sends the record's  keys, values pairs
    return [normalize_record(list(head.items()))] + normalize_numbers(tail, fields, precision)



# # 6 — clean_strings

def clean_strings(records, fields):

    def clean_record(items):
        if not items:
            return {}
        k, v = items[0]
        rest = clean_record(items[1:])
        if k in fields and isinstance(v, str):
            return {k: v.strip().title(), **rest}
        else:
            return {k: v, **rest}

    if not records:
        return []

    head, *tail = records
    return [clean_record(list(head.items()))] + clean_strings(tail, fields)

In [3]:
# CSV Loader and Saver
def load_csv(filename):
    return pd.read_csv(filename)


def save_csv(filename, records):
    if not records:
        return
    with open(filename, "w", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=records[0].keys())
        writer.writeheader()
        writer.writerows(records)

In [4]:
copy_of_original_dataset = load_csv("../retail_store_sales.csv")
copy_of_original_df1 = pd.DataFrame(copy_of_original_dataset)
copy_of_original_df1.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,4/8/2024,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True
2,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,5/7/2022,NaN
3,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,10/2/2022,False
4,TXN_7482416,CUST_09,Patisserie,NaN,NaN,10.0,200.0,Credit Card,Online,11/30/2023,NaN


In [5]:
len(copy_of_original_df1)

12574

In [6]:
# let's check the number of empty values in fields
copy_of_original_df1.isna().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [7]:
# replace NaN with None for consistency
copy_of_original_df = copy_of_original_df1.where(pd.notna(copy_of_original_df1), None)
# changing the dataframe into a dictionary to be proccessed by the functions
copy_of_original_records = copy_of_original_df.to_dict(orient="records")


In [8]:
"""
testing remove_missing() function :) 
"""
removed_missing_records= remove_missing(copy_of_original_records,
['Item', 'Price Per Unit', 'Quantity', 'Total Spent', 'Discount Applied'])
# changing the dictionary back into dataframe
removed_missing_df = pd.DataFrame(removed_missing_records)
removed_missing_df.isna().sum()

Transaction ID      0
Customer ID         0
Category            0
Item                0
Price Per Unit      0
Quantity            0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
Discount Applied    0
dtype: int64

In [9]:
removed_missing_df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,4/8/2024,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True
2,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,10/2/2022,False
3,TXN_3652209,CUST_07,Food,Item_1_FOOD,5.0,8.0,40.0,Credit Card,In-store,6/10/2023,True
4,TXN_9728486,CUST_23,Furniture,Item_16_FUR,27.5,1.0,27.5,Credit Card,In-store,4/26/2023,False


In [10]:
len(removed_missing_df)

7578

In [13]:
"""
testing fill_missing() function :) 
"""
removed_missing_prices_records= remove_missing(copy_of_original_records,
['Price Per Unit'])
price = [r["Price Per Unit"] for r in removed_missing_prices_records]
removed_missing_quantity_records= remove_missing(copy_of_original_records,
['Quantity'])
quantity = [r["Quantity"] for r in removed_missing_quantity_records]
removed_missing_spent_records= remove_missing(copy_of_original_records,
['Total Spent'])
spent = [r["Total Spent"] for r in removed_missing_spent_records]
defaults = {
    "Item": "Unknown",
    "Price Per Unit": round(mean(price), 1),
    "Quantity": mean(quantity),
    "Total Spent": mean(spent),
    "Discount Applied": False
}
filled_missing_records = fill_missing(copy_of_original_records, defaults)
filled_missing_df = pd.DataFrame(filled_missing_records)
filled_missing_df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,4/8/2024,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True
2,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,5/7/2022,False
3,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,10/2/2022,False
4,TXN_7482416,CUST_09,Patisserie,Unknown,23.4,10.0,200.0,Credit Card,Online,11/30/2023,False


In [14]:
len(filled_missing_df)

12574

In [15]:
filled_missing_df.isna().sum()

Transaction ID      0
Customer ID         0
Category            0
Item                0
Price Per Unit      0
Quantity            0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
Discount Applied    0
dtype: int64

In [16]:
"""
testing standardize_dates() function :) 
"""
standardized_records = standardize_dates(copy_of_original_records, "Transaction Date")
standardize_df = pd.DataFrame(standardized_records)
standardize_df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,None
3,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False
4,TXN_7482416,CUST_09,Patisserie,None,NaN,10.0,200.0,Credit Card,Online,2023-11-30,None


In [17]:
"""
testing normalize_numbers() function :) 
"""
normalized_numbers_records = normalize_numbers(copy_of_original_records, ['Price Per Unit', 'Total Spent'], 0)
normalized_numbers_df = pd.DataFrame(normalized_numbers_records)
normalized_numbers_df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.0,10.0,185.0,Digital Wallet,Online,4/8/2024,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True
2,TXN_9458126,CUST_06,Beverages,Item_16_BEV,28.0,9.0,248.0,Credit Card,Online,5/7/2022,None
3,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.0,7.0,88.0,Digital Wallet,Online,10/2/2022,False
4,TXN_7482416,CUST_09,Patisserie,None,NaN,10.0,200.0,Credit Card,Online,11/30/2023,None


In [18]:
"""
testing clean_strings() function :) 
"""
cleaned_string_records = clean_strings(copy_of_original_records, ['Category', 'Payment Method'])
cleaned_string_df = pd.DataFrame(cleaned_string_records)
cleaned_string_df.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,4/8/2024,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True
2,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,5/7/2022,None
3,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,10/2/2022,False
4,TXN_7482416,CUST_09,Patisserie,None,NaN,10.0,200.0,Credit Card,Online,11/30/2023,None


In [19]:
"""
testing save_csv() function :) 
"""
save_csv("removed_missing_data.csv", removed_missing_records)

In [20]:
created_dataset = load_csv("removed_missing_data.csv")
created_df = pd.DataFrame(created_dataset)
len(created_df)

7578

In [21]:
import operator
import csv
from itertools import groupby
from functools import reduce

In [22]:
# 1. filter functions and helper function create condition
def create_condition(field_name, op_func, target_value):
    def condition(record):
        return op_func(record.get(field_name), target_value)

    return condition


def filter_records(records, condition):
    # returns a new list
    return list(filter(condition, records))


# ------------------------------- 2. add column funtion------------------------------------------------------

def add_column(records, column_name, compute_fn):
    # returns a new list with new columns
    return list(map(lambda r: {**r, column_name: compute_fn(r)}, records))


# the new columns calculations
# 1. calculate expected total
def calculate_expected_total(records):
    price = records.get("Price Per Unit", 0)
    quantity = records.get("Quantity", 0)
    return round(price * quantity)


# 2. calculate_taxe
def calculate_tax(records):
    total_spent = records.get("Total Spent", 0)
    return round(total_spent * 0.25)


# -----------------------------------------------------3. group by function------------------------------------
def group_by(records, key):
    sorted_data = sorted(records, key=key)

    # pipeline

    # 1. groupby will group data based on the given key (key ,group)
    # 2. map will return list  (key,list(group))
    # 3. dicionary will make it into a dictionary  again like that {key: v}
    return dict(map(
        lambda item: (item[0], list(item[1])),  # item[0] is key and item[1] is iterator
        groupby(sorted_data, key=key)
    ))


# --------------------------------------------------4. aggregate function------------------------------------------
def aggregate(groups, agg_fn):
    # aggregate based on teh grouped items
    return dict(map(
        lambda item: (item[0], agg_fn(item[1])),
        groups.items()
    ))


def sum_revenue(records):
    # [{'Total': 10}, {'Total': 20}] into [10, 20]
    amounts = map(lambda r: r['Total Spent'], records)
    # [10, 20] into 30 using addition
    return reduce(operator.add, amounts, 0)


# -----------------------------------------------------5. drop column------------------------------------------------
def drop_column(records, name):
    return list(map(
        lambda r: dict(filter(
            lambda item: item[0] != name,
            r.items()
        )), records
    ))


# ---------------------------------------------------6. map function ---------------------------------------------------
def map_records(records, transform_fn):
    return list(map(transform_fn, records))


def fix_types(records):
    new_record = records.copy()

    # Internal helper to safely convert
    def safe_float(value):
        if value is None or value == '':
            return 0.0
        try:
            return float(value)
        except ValueError:
            return 0.0

    new_record['Total Spent'] = safe_float(records.get('Total Spent'))
    new_record['Quantity'] = safe_float(records.get('Quantity'))
    new_record['Price Per Unit'] = safe_float(records.get('Price Per Unit'))
    return new_record

In [23]:
# 2. TEST MAP (Fix Types)
# ---------------------------------------------------------
# We MUST fix types first because CSV loads numbers as strings like "18.5"
data_typed = map_records(filled_missing_records, fix_types)
data_typed_df = pd.DataFrame(data_typed)
print(f"2. MAP: Fixed types. Sample 'Total Spent': {data_typed[0]['Total Spent']} (Type: {type(data_typed[0]['Total Spent'])})")

2. MAP: Fixed types. Sample 'Total Spent': 185.0 (Type: <class 'float'>)


In [24]:
# 3. TEST FILTER (Total Spent > 0)
# ---------------------------------------------------------
positive_condition = create_condition('Total Spent', operator.gt, 0)
data_filtered = filter_records(data_typed, positive_condition)
print(f"3. FILTER: Kept {len(data_filtered)} positive records (Dropped {len(data_typed) - len(data_filtered)}).")

3. FILTER: Kept 12574 positive records (Dropped 0).


In [25]:
# 4. TEST ADD COLUMN (Expected Total & Tax)
# ---------------------------------------------------------
data_w_expected = add_column(data_filtered, 'Expected Total', calculate_expected_total)
data_w_tax = add_column(data_w_expected, 'Tax', calculate_tax)
data_w_tax_df = pd.DataFrame(data_w_tax)
data_w_tax_df.head()
# print(f"4. ADD COLUMN: Added 'Expected Total' & 'Tax'. Sample Record 1: Expected={data_w_tax[0]['Expected Total']}, Tax={data_w_tax[0]['Tax']}")

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied,Expected Total,Tax
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,4/8/2024,True,185,46
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True,261,65
2,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,5/7/2022,False,248,62
3,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,10/2/2022,False,88,22
4,TXN_7482416,CUST_09,Patisserie,Unknown,23.4,10.0,200.0,Credit Card,Online,11/30/2023,False,234,50


In [26]:
# 5. TEST DROP COLUMN (Remove 'Item')
# ---------------------------------------------------------
data_dropped = drop_column(data_w_tax, 'Item')
data_dropped_df = pd.DataFrame(data_dropped)
data_dropped_df.head()
# print(f"5. DROP COLUMN: Removed 'Item'. Remaining Keys: {list(data_dropped[0].keys())}")

,Transaction ID,Customer ID,Category,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied,Expected Total,Tax
0,TXN_6867343,CUST_09,Patisserie,18.5,10.0,185.0,Digital Wallet,Online,4/8/2024,True,185,46
1,TXN_3731986,CUST_22,Milk Products,29.0,9.0,261.0,Digital Wallet,Online,7/23/2023,True,261,65
2,TXN_9458126,CUST_06,Beverages,27.5,9.0,247.5,Credit Card,Online,5/7/2022,False,248,62
3,TXN_4575373,CUST_05,Food,12.5,7.0,87.5,Digital Wallet,Online,10/2/2022,False,88,22
4,TXN_7482416,CUST_09,Patisserie,23.4,10.0,200.0,Credit Card,Online,11/30/2023,False,234,50


In [27]:
# 6. TEST GROUP BY (Category)
# ---------------------------------------------------------
# We group by Category. Lambda extracts the key.
grouped_data = group_by(data_dropped, lambda r: r['Category'])
print(f"6. GROUP BY: Found {len(grouped_data)} categories: {list(grouped_data.keys())}")

6. GROUP BY: Found 8 categories: ['Beverages', 'Butchers', 'Computers and electric accessories', 'Electric household essentials', 'Food', 'Furniture', 'Milk Products', 'Patisserie']


In [28]:
# 7. TEST AGGREGATE (Sum Revenue)
# ---------------------------------------------------------
final_report = aggregate(grouped_data, sum_revenue)
print("\n--- 7. FINAL AGGREGATE REPORT (Revenue per Category) ---")
import pprint
pprint.pprint(final_report)


--- 7. FINAL AGGREGATE REPORT (Revenue per Category) ---
{'Beverages': 206253.34695071031,
 'Butchers': 217410.50676691753,
 'Computers and electric accessories': 201194.94511278218,
 'Electric household essentials': 213537.9862155391,
 'Food': 205314.44511278218,
 'Furniture': 203867.5478696744,
 'Milk Products': 189317.84695071034,
 'Patisserie': 193445.90401002532}


In [29]:
#--------- Mean -----------
def mean(values):
    values = list(values)
    return sum(values) / len(values) if values else 0  # return mean if not empty else 0


#--------- Median ----------
def median(values):
    values = sorted(values)
    length = len(values)

    if length == 0:
        return 0
    
    mid = length // 2
    if length % 2 == 1:  # length is odd number 
        return values[mid]
    else:
        return (values[mid - 1 ] + values[mid]) / 2
    

#--------- Variance ---------
def variance(values):
    values = list(values)
    if len(values) == 0:
        return 0

    m = mean(values)   
    squared_diffs = map(lambda v: (v - m) ** 2, values)
    return sum(squared_diffs) / len(values)

#----------- Summaries ---------
def summaries(records, fields):
    extract_values = lambda field: list(
        map(float,
            filter(lambda v: v not in ("", None),
                   map(lambda r: r.get(field), records)))
    )
# records = [
#     {"Price": 10, "Quantity": 3},
#     {"Price": 15, "Quantity": 5},
#     {"Price": "", "Quantity": 2},
# ]
# list(map(lambda r: r.get("Price"), records))
# # Output: [10, 15, '']
# Output: [10.0, 15.0]

    compute_summary = lambda field: {
        "mean": mean(extract_values(field)),
        "median": median(extract_values(field)),
        "variance": variance(extract_values(field))
    }

    return dict(map(lambda f: (f, compute_summary(f)), fields)) # build dictionary for all fields 




#----------- Correlation ---------
def correlation(values_x, values_y):
    x = list(values_x)
    y = list(values_y)

    if not x or len(x) != len(y):
        return 0
    
    mean_x = sum(x) / len(x)
    mean_y = sum(y) / len(y)

    # numerator: sum((xi - mean_x) * (yi - mean_y) for xi, yi in zip(x, y))
    numerator = sum(map(lambda pair: (pair[0] - mean_x) * (pair[1] - mean_y),
                        zip(x, y)))
    # denominator: sqrt(sum((xi - mean_x)^2) * sum((yi - mean_y)^2))
    sum_sq_x = sum(map(lambda xi: (xi - mean_x) ** 2, x))
    sum_sq_y = sum(map(lambda yi: (yi - mean_y) ** 2, y))
    denominator = math.sqrt(sum_sq_x * sum_sq_y)

    return numerator / denominator if denominator != 0 else 0

#----- trend analysis -----------------
def compute_trend(values):
    values = list(values)
    if len(values) < 2:
        return []

    # Zip each value with its previous value
    pairs = zip(values[:-1], values[1:]) # all value except the last , all value except the first

    # [15, 20, 33 ] -> (15, 20) -> (20-15)/15 = 0.33 , then complete the list : 0.33 , 0.65,....
    return list(map(lambda pair: (pair[1] - pair[0]) / pair[0] if pair[0] != 0 else 0, pairs))

In [31]:
fields = ["Price Per Unit", "Quantity", "Total Spent"]

# Mean
prices = [r["Price Per Unit"] for r in filled_missing_records]
print("Mean Price:", mean(prices))


# Median
print("Median Quantity:", median([r["Quantity"] for r in filled_missing_records]))


# Variance
print("Variance Total Spent:", variance([r["Total Spent"] for r in filled_missing_records]))

# Summaries
print("\nSummaries for all fields:")
summary_result = summaries(filled_missing_records, fields)
for field, stats in summary_result.items():
    print(f"{field}: {stats}")

# Correlation
quantity = [r["Quantity"] for r in filled_missing_records]
total_spent = [r["Total Spent"] for r in filled_missing_records]
print("\nCorrelation between Quantity and Total Spent:", correlation(quantity, total_spent))


# Compute Trend
sales_over_time = [r["Total Spent"] for r in filled_missing_records]
trend_values = compute_trend(sales_over_time)
# Print only first 5 values
print("Trend (percent change) of Total Spent (first 5 values):", trend_values[:5])


Mean Price: 23.36771114999199
Median Quantity: 5.536675020885547
Variance Total Spent: 8545.848120852228

Summaries for all fields:
Price Per Unit: {'mean': 23.36771114999199, 'median': 23.4, 'variance': 109.83265394678413}
Quantity: {'mean': 5.536675020885547, 'median': 5.536675020885547, 'variance': 7.7741688934173725}
Total Spent: {'mean': 129.65981620718296, 'median': 112.5, 'variance': 8545.848120852228}

Correlation between Quantity and Total Spent: 0.7120449481419476
Trend (percent change) of Total Spent (first 5 values): [0.41081081081081083, -0.05172413793103448, -0.6464646464646465, 1.2857142857142858, -0.8]
